In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features, get_train_test_with_excluded_columns
from helper.clfmodel_functions import tune_model, forward_feat_selection_hypertuning
from helper.fairness_functions import print_male_female_metrics, get_male_female_data

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'


### Loading the cleaned dataset

In [2]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

### Feature Engineering (encoding) & Train-Test Split

In [3]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
X_male, X_female = get_male_female_data(X, False)
# Encoding the features and target
X_encoded, y_encoded = encode_all_features(X, y, [])
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [4]:
X_train

,age,education,workinghours,ability to speak english,marital status_Divorced,marital status_Husband,marital status_Never married,marital status_Separated,marital status_Widowed,marital status_Wife,...,occupation_Service/Hospitality,occupation_Transport,workclass_governmental,workclass_no paid work,workclass_private,workclass_self employed,gave birth this year_No,gave birth this year_Yes,sex_Female,sex_Male
6317,22,16,36,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
740,61,22,40,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
3781,48,16,40,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
7850,62,18,65,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
2963,53,19,44,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,22,19,25,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,1,0
5191,24,16,28,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,1,0
5390,35,16,40,0,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,1
860,23,20,40,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,1


### Model

Here, we quickly train and evaluate a Decision Tree model with random parameters for demonstration.

In [5]:
# Decision Tree model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_preds = dt_model.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_preds)

print(classification_report(y_test, dt_preds))
print("Decision Tree Accuracy:", dt_accuracy)

              precision    recall  f1-score   support

           0       0.78      0.79      0.78      1175
           1       0.59      0.58      0.59       625

    accuracy                           0.72      1800
   macro avg       0.69      0.68      0.68      1800
weighted avg       0.71      0.72      0.72      1800

Decision Tree Accuracy: 0.7161111111111111


### Feature Importance using the model itself

In [6]:
# Extract feature importances
feature_importances = dt_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [7]:
feature_importance_df

,Feature,Importance
0,age,0.282893
1,education,0.163422
2,workinghours,0.141988
5,marital status_Husband,0.130882
18,occupation_Management/Business,0.020010
30,workclass_private,0.018155
25,"occupation_Science, Engineering, Technology",0.017908
28,workclass_governmental,0.016920
21,occupation_Production/Assembly,0.016158
24,occupation_Sales,0.014524


### Hyperparameter tuning

Here, we define the base parameter grid for our hyperparameter tuning function.

In [8]:
param_grid = {
        'criterion': ['gini', 'entropy'],
        'splitter': ['best', 'random'],
        'max_depth': [None, 5, 10, 15, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': [None, 'sqrt', 'log2'],
        'random_state': [42]
    }

Below, we tune the hyperparameters of the Decision Tree model using the defined parameter grid and using all features.


In [9]:
best_params, best_model, best_accuracy = tune_model(DecisionTreeClassifier(), X_train, y_train, X_test, y_test, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

dt_preds = best_model.predict(X_test)
print(classification_report(y_test, dt_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/dt_model_sexincl_noffs_tuned_unfair.joblib')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'random_state': 42, 'splitter': 'best'}
Best Model: DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=2,
                       min_samples_split=5, random_state=42)
Best Model Accuracy: 0.7722222222222223
              precision    recall  f1-score   support

           0       0.81      0.85      0.83      1175
           1       0.69      0.62      0.65       625

    accuracy                           0.77      1800
   macro avg       0.75      0.74      0.74      1800
weighted avg       0.77      0.77      0.77      1800

Fairness Metrics:
Male FPR: 0.18335684062059238
Male TPR: 0.6653306613226453
Female FPR: 0.0944206008583691
Female TPR: 0.4523809523809524

Disparate Impact (DI): 0.446
Discrimination Score (DS): -0.212
Equal Opportunity Difference (EO): 0.213


Now the same but with fairness constraints.

In [10]:
best_params, best_model, best_accuracy = tune_model(DecisionTreeClassifier(), X_train, y_train, X_test, y_test, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

dt_preds = best_model.predict(X_test)
print(classification_report(y_test, dt_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/dt_model_sexincl_noffs_tuned_fair.joblib')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 5, 'random_state': 42, 'splitter': 'random'}
Best Model: DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=4,
                       min_samples_split=5, random_state=42, splitter='random')
Best Model Accuracy: 0.7622222222222222
              precision    recall  f1-score   support

           0       0.78      0.88      0.83      1175
           1       0.71      0.53      0.61       625

    accuracy                           0.76      1800
   macro avg       0.75      0.71      0.72      1800
weighted avg       0.76      0.76      0.75      1800

Fairness Metrics:
Male FPR: 0.13540197461212977
Male TPR: 0.56312625250501
Female FPR: 0.08583690987124463
Female TPR: 0.4126984126984127

Disparate Impact (DI): 0.498
Discrimination Score (DS): -0.157
Equal Opportunity Dif

Below, we tune the hyperparameters of the Decision Tree model using the defined parameter grid and using a **subset of features**. We exclude the columns 'age', 'ability to speak english', and 'workclass'.

In [11]:
columns_to_exclude = ['age', 'ability to speak english', 'workclass']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(DecisionTreeClassifier(), X_train_, y_train_, X_test_, y_test_, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

dt_preds = best_model.predict(X_test_)
print(classification_report(y_test_, dt_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/dt_model_sexincl_noffs_tuned_unfair_aawexcl.joblib')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'random_state': 42, 'splitter': 'best'}
Best Model: DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=10,
                       random_state=42)
Best Model Accuracy: 0.775
              precision    recall  f1-score   support

           0       0.81      0.85      0.83      1175
           1       0.70      0.63      0.66       625

    accuracy                           0.78      1800
   macro avg       0.75      0.74      0.75      1800
weighted avg       0.77      0.78      0.77      1800

Fairness Metrics:
Male FPR: 0.20451339915373765
Male TPR: 0.6973947895791583
Female FPR: 0.055793991416309016
Female TPR: 0.3412698412698413

Disparate Impact (DI): 0.286
Discrimination Score (DS): -0.292
Equal Opportunity Difference (EO): 0.356
Equalized Odds (EOdds): 0.149

Now the same but with fairness constraints.

In [12]:
columns_to_exclude = ['age', 'ability to speak english', 'workclass']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(DecisionTreeClassifier(), X_train_, y_train_, X_test_, y_test_, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

dt_preds = best_model.predict(X_test_)
print(classification_report(y_test_, dt_preds))

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/dt_model_sexincl_noffs_tuned_fair_aawexcl.joblib')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 15, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'random_state': 42, 'splitter': 'best'}
Best Model: DecisionTreeClassifier(criterion='entropy', max_depth=15, max_features='log2',
                       min_samples_split=10, random_state=42)
Best Model Accuracy: 0.7538888888888889
              precision    recall  f1-score   support

           0       0.78      0.86      0.82      1175
           1       0.68      0.55      0.61       625

    accuracy                           0.75      1800
   macro avg       0.73      0.71      0.71      1800
weighted avg       0.75      0.75      0.75      1800

Fairness Metrics:
Male FPR: 0.1607898448519041
Male TPR: 0.5971943887775552
Female FPR: 0.10515021459227468
Female TPR: 0.373015873015873

Disparate Impact (DI): 0.475
Discrimination Score (DS): -0.179
Equal Opportunity Difference (EO): 0.

#### Forward Feature Selection with Hyperparameter Tuning

Because hyperparameter tuning results can vary depending on the chosen subset of features, we will use forward feature selection with hyperparameter tuning to find the best subset of features and hyperparameters for the model.

In [13]:
# Forward feature selection with hyperparameter tuning
best_subset, best_params, best_score = forward_feat_selection_hypertuning(DecisionTreeClassifier(), param_grid, X_train, y_train, X_test, y_test)

print("Best subset of features:", best_subset)
print("Best hyperparameters:", best_params)
print("Best model accuracy:", best_score)

# Use the best subset and best hyperparameters for final model
final_model_allf_ffs = DecisionTreeClassifier(**best_params)
final_model_allf_ffs.fit(X_train[best_subset], y_train)
final_model_preds = final_model_allf_ffs.predict(X_test[best_subset])
final_model_accuracy = accuracy_score(y_test, final_model_preds)

print(classification_report(y_test, final_model_preds))

print_male_female_metrics(final_model_allf_ffs, X, X_male, X_female, X_test[best_subset], y_test)
save_model(final_model_allf_ffs, '../output/saved_models/dt_model_sexincl_ffs_tuned_unfair.joblib')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Best subset: ['education']
Remaining features: [['age'], ['workinghours'], ['ability to speak english'], ['workclass_governmental', 'workclass_no paid work', 'workclass_private', 'workclass_self employed'], ['marital status_Divorced', 'marital status_Husband', 'marital status_Never married', 'marital status_Separated', 'marital status_Widowed', 'marital status_Wife'], ['occupation_Construction/Extraction', 'occupation_Co

Now the same but with fairness constraints.

In [14]:
# Forward feature selection with hyperparameter tuning
best_subset, best_params, best_score = forward_feat_selection_hypertuning(DecisionTreeClassifier(), param_grid, X_train, y_train, X_test, y_test, ensure_fairness=True)

print("Best subset of features:", best_subset)
print("Best hyperparameters:", best_params)
print("Best model accuracy:", best_score)

# Use the best subset and best hyperparameters for final model
final_model_allf_ffs = DecisionTreeClassifier(**best_params)
final_model_allf_ffs.fit(X_train[best_subset], y_train)
final_model_preds = final_model_allf_ffs.predict(X_test[best_subset])
final_model_accuracy = accuracy_score(y_test, final_model_preds)

print(classification_report(y_test, final_model_preds))

print_male_female_metrics(final_model_allf_ffs, X, X_male, X_female, X_test[best_subset], y_test)
save_model(final_model_allf_ffs, '../output/saved_models/dt_model_sexincl_ffs_tuned_fair.joblib')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Best subset: ['sex_Female', 'sex_Male', 'education']
Remaining features: [['age'], ['workinghours'], ['ability to speak english'], ['workclass_governmental', 'workclass_no paid work', 'workclass_private', 'workclass_self employed'], ['marital status_Divorced', 'marital status_Husband', 'marital status_Never married', 'marital status_Separated', 'marital status_Widowed', 'marital status_Wife'], ['occupation_Construction/Extraction', 'occupation_Counseling/Mental Health Services', 'occ

#### Removing sex-related columns

Below we do Forward feature selection with hyperparameter tuning, but we exclude all sex-related columns.

In [15]:
# Forward feature selection with hyperparameter tuning
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_subset, best_params, best_score = forward_feat_selection_hypertuning(DecisionTreeClassifier(), param_grid, X_train_, y_train_, X_test_, y_test_)

print("Best subset of features:", best_subset)
print("Best hyperparameters:", best_params)
print("Best model accuracy:", best_score)

# Use the best subset and best hyperparameters for final model
final_model_sexexcl_ffs = DecisionTreeClassifier(**best_params)
final_model_sexexcl_ffs.fit(X_train_[best_subset], y_train_)
final_model_preds = final_model_sexexcl_ffs.predict(X_test_[best_subset])
final_model_accuracy = accuracy_score(y_test_, final_model_preds)

print(classification_report(y_test_, final_model_preds))

print_male_female_metrics(final_model_sexexcl_ffs, X, X_male, X_female, X_test_[best_subset], y_test_)
save_model(final_model_sexexcl_ffs, '../output/saved_models/dt_model_sexexcl_ffs_tuned_unfair.joblib')

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Fitting 5 folds for each of 540 candidates, totalling 2700 fits
Best subset: ['education']
Remaining features: [['age'], ['workinghours'], ['ability to speak english'], ['workclass_governmental', 'workclass_no paid work', 'workclass_private', 'workclass_self employed'], ['occupation_Construction/Extraction', 'occupation_Counseling/Mental Health Services', 'occupation_Education', 'occupation_Entertainment', 'occupation_Farming, Fishing, Forestry', 'occupation_Finance/Accounting', 'occupation_Healthcare/Medical Services', 'occupation_Legal Services', 'occupation_Management/Business', 'occupation_Military Services', 'occupation_Office/Administrative Support', 'occupation_Pro

### AdaBoost Classifier

In [ ]:
# Define the parameter grid for AdaBoost
param_grid = {
    'algorithm': ['SAMME', 'SAMME.R'],
    'n_estimators': [50, 75, 100, 150],
    'learning_rate': [0.1, 0.4, 0.5, 1.0],
    'estimator': [None, final_model_allf_ffs], # None means default estimator = DecisionTreeClassifier
    'random_state': [42]
}

Below, we tune the hyperparameters of the AdaBoost model using the defined parameter grid and using all features.

In [ ]:
best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train, y_train, X_test, y_test, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/dt_model_sexincl_noffs_tuned_unfair_ada.joblib')

Now the same but with fairness constraints.

In [ ]:
best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train, y_train, X_test, y_test, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

print_male_female_metrics(best_model, X, X_male, X_female, X_test, y_test)
save_model(best_model, '../output/saved_models/dt_model_sexincl_noffs_tuned_fair_ada.joblib')

Below, we tune the hyperparameters of the AdaBoost model using the defined parameter grid and using a **subset of features**. We exclude the sex-related columns.

In [ ]:
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train_, y_train_, X_test_, y_test_, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/dt_model_sexexcl_noffs_tuned_unfair_ada.joblib')

Now the same but with fairness constraints.

In [ ]:
columns_to_exclude = ['sex', 'gave birth this year', 'marital status']
X_train_, X_test_, y_train_, y_test_, X_male, X_female = get_train_test_with_excluded_columns(data, columns_to_exclude)

best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train_, y_train_, X_test_, y_test_, param_grid, ensure_fairness=True)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

print_male_female_metrics(best_model, X, X_male, X_female, X_test_, y_test_)
save_model(best_model, '../output/saved_models/dt_model_sexexcl_noffs_tuned_fair_ada.joblib')